# **General Matrix Multiplication (GEMM) Optimization**

## **Pre-requistes**

1. The code adds the CUDA 12.6 compiler's location to the system's PATH environment variable and then displays the version information for NVIDIA's CUDA compiler (nvcc), which is used for GPU programming.

In [22]:
import os
os.environ["PATH"] += ":/usr/local/cuda-12.6/bin" # Add your path to CUDA
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Tue_Oct_29_23:50:19_PDT_2024
Cuda compilation tools, release 12.6, V12.6.85
Build cuda_12.6.r12.6/compiler.35059454_0


2. Device details

In [23]:
! nvidia-smi

Mon Dec  9 19:55:20 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.59                 Driver Version: 556.13         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...    On  |   00000000:01:00.0  On |                  N/A |
| N/A   47C    P5             14W /   95W |     434MiB /   6144MiB |     66%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

3. Function to execute object files repeatedly and print the average kernel runtime.

In [24]:
import re
import subprocess
import statistics
import time  # Added for sleep functionality

def run_gemm(executable_path, choice, m, n, k, runs=10, sleep_time=2):  # Added sleep_time parameter
    times = []
    
    for i in range(runs):
        result = subprocess.run([executable_path, str(choice), str(m), str(n), str(k)],
                              capture_output=True, text=True)
        print(f"Output {i+1} - {result.stdout}") 
        match = re.search(r'CUDA kernel time: (\d+\.\d+)', result.stdout)
        if match:
            cuda_time = float(match.group(1))
            times.append(cuda_time)
        else:
            print(f"Warning: No time found in output: {result.stdout}")
        
        # Add sleep between runs, except for the last run
        if i < runs - 1:
            time.sleep(sleep_time)
            print(f"Sleeping for {sleep_time} seconds...")
    
    # Calculate and print statistics
    mean_time = statistics.mean(times)
    std_dev = statistics.stdev(times) if len(times) > 1 else 0
    min_time = min(times) if times else 0
    max_time = max(times) if times else 0
    
    print(f"\nStatistics:")
    print(f"Mean: {mean_time:.2f} ms")
    print(f"Std Dev: {std_dev:.2f} ms")
    print(f"Min: {min_time:.2f} ms")
    print(f"Max: {max_time:.2f} ms")
    
    return mean_time

## **1 - Naive GEMM Kernel**

<div style="text-align: center;">
  <img src="./images/naive_kernel_mul.png" alt="Naive GEMM Multiplication" width="800">
</div>

This diagram shows a naive GEMM (General Matrix Multiplication) kernel implementation using threads. Each thread accesses matrix elements based on its ID: `x = blockDim.x * blockIdx.x + threadIdx.x` and `y = blockDim.y * blockIdx.y + threadIdx.y`. Within the B matrix, threads in a warp access the same values (broadcast), while in the A matrix, threads access non-consecutive memory locations (non-coalesced memory access), which is inefficient. The C matrix shows how different threads (0,0), (0,1), (0,2) etc., compute their respective output elements through these memory access patterns.

<div style="text-align: center;">
  <img src="./images/naive_kernel_memory_access.png" alt="Naive Kernel Memory Access" width="800">
</div>

This diagram illustrates a memory access pattern issue in GPU computing. It shows two warps (groups of threads) accessing memory in a non-coalesced pattern, meaning threads access scattered memory locations rather than consecutive ones. Each warp requires 4x32B loads (8 loads total), which is inefficient. The crossing lines between thread indices and memory locations visualize this scattered access pattern. This non-optimal memory access results in performance penalties because too many separate load operations are needed to execute each warp.

### **a. Compile the code**

The file being executed is [./src/01_naive_gemm.cu](./src/01_naive_gemm.cu)

In [25]:
! nvcc -lineinfo -g -o ./src/01_naive_gemm ./src/run.cu -lcublas -lnvToolsExt

### **b. Execute the object file multiple times**

In [26]:
naive_gemm_time = run_gemm("./src/01_naive_gemm", 1, 4096, 4096, 4096)
print(f"Average Naive GEMM time: {naive_gemm_time}")

Output 1 - Naive GEMM Kernel:
CUDA kernel time: 1571.1896 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 2 - Naive GEMM Kernel:
CUDA kernel time: 1579.1021 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 3 - Naive GEMM Kernel:
CUDA kernel time: 1638.3568 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 4 - Naive GEMM Kernel:
CUDA kernel time: 1577.6754 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 5 - Naive GEMM Kernel:
CUDA kernel time: 1588.3717 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 6 - Naive GEMM Kernel:
CUDA kernel time: 1564.9207 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 7 - Naive GEMM Kernel:
CUDA kernel time: 1548.5822 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 8 - Naive GEMM Kernel:
CUDA kernel time: 1578.2319 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 9 - Naive GEMM Kernel:
CUDA kernel time: 1571.5762 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 10 - Naive G

### **c. Generate report for the kernel**

In [27]:
! ncu -f --set full -o ./profiles/01_naive_gemm ./src/01_naive_gemm 1 4096 4096 4096

==PROF== Connected to process 19082 (/home/darshith/code/cuda-gemm-optimization/src/01_naive_gemm)
==PROF== Profiling "ampere_sgemm_128x64_nn" - 0: 0%
==WARNING== Launching the workload is taking more time than expected. If this continues to hang, terminate the profile and re-try by profiling the range of all related launches using '--replay-mode range'. See https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#replay for more details.
....50%....100% - 38 passes
Naive GEMM Kernel:
==PROF== Profiling "gemmNaive" - 1: 0%
==WARNING== Launching the workload is taking more time than expected. If this continues to hang, terminate the profile and re-try by profiling the range of all related launches using '--replay-mode range'. See https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#replay for more details.
....50%....100% - 38 passes
CUDA kernel time: 252408.7344 ms
Results match : Yes 
==PROF== Disconnected from process 19082
==PROF== Report: /home/darshith/code/cud

### **d. Report**

**Speed of Light Throughput:**

<div style="text-align: center;">
  <img src="./images/naive_sol.png" alt="Naive SOL" width="1500">
</div>

The GPU throughput graph shows significant memory utilization (approximately 90% of SOL) while compute utilization is relatively low (around 15% of SOL). This indicates the naive GEMM implementation is heavily memory-bound, primarily constrained by memory bandwidth rather than computational capacity.

**Roofline:**

<div style="text-align: center;">
  <img src="./images/naive_roofline.png" alt="Naive Roofline" width="1500">
</div>

The roofline plot reveals this naive GEMM implementation is significantly underperforming compared to the hardware's theoretical peak performance, achieving around 51.8 GFLOP/s with an arithmetic intensity of 14.58 FLOP/byte. The large gap between the actual performance point and the peak performance line suggests substantial room for optimization through techniques like better memory access patterns, cache utilization, and vectorization.

**Memory Workload Analysis:**

<div style="text-align: center;">
  <img src="./images/naive_memory_workload_analysis.png" alt="Naive Memory Workload Analysis" width="2000">
</div>

This GEMM kernel shows suboptimal memory access patterns. The L1TEX hit rate is high at 99.11%, but the memory throughput is quite low at 3.56 GB/s, suggesting memory bandwidth isn't fully utilized. Both global load and store access patterns are inefficient, with only 4.0 bytes out of 32 bytes per sector being utilized per thread (indicated by the 86.62% speedup estimate). This points to non-coalesced memory accesses and potential thread stride issues that are preventing optimal memory bandwidth utilization, despite good cache performance.

## **2 - Coalesced Memory GEMM Kernel**

<div style="text-align: center;">
  <img src="./images/coalesced_memory_mul.png" alt="Coalesced Memory Multiplication" width="800">
</div>

This diagram shows an optimized memory coalesced GEMM (General Matrix Multiplication) kernel design. Unlike the naive version, threads access consecutive memory locations in matrix B, enabling memory coalescing and better performance. For matrix A, all threads within a warp access the same values (broadcast). The coordinates are calculated as: `x = blockIdx.x * BLOCK_SIZE + (threadIdx.x / BLOCK_SIZE)` for matrix A's row access, and `y = blockIdx.y * BLOCK_SIZE + (threadIdx.y % BLOCK_SIZE)` for matrix B's column access. Threads (0,0), (0,1), and (0,2) are grouped in the same warp to optimize memory access patterns.

<div style="text-align: center;">
  <img src="./images/coalesced_memory_access.png" alt="Coalesced Memory Access" width="800">
</div>

This diagram shows an optimized memory coalesced access pattern where threads within each warp (Warp-0 and Warp-1) access consecutive memory locations. Each warp now requires only 2x32B loads (4 loads total), half of what was needed in the non-coalesced version. The straight vertical lines from thread indices to memory locations indicate efficient coalesced memory access, improving performance by reducing the number of required load operations.

### **a. Compile the code**

The file being executed is [./src/02_memory_coalesced_gemm.cu](./src/02_memory_coalesced_gemm.cu)

In [28]:
!nvcc -lineinfo -g -o ./src/02_memory_coalesced_gemm ./src/run.cu -lcublas -lnvToolsExt

### **b. Execute the object file multiple times**

In [29]:
memory_coalesced_gemm_time = run_gemm("./src/02_memory_coalesced_gemm", 2, 4096, 4096, 4096)
print(f"Average Global Memory Coalesced GEMM time: {memory_coalesced_gemm_time}")

Output 1 - Global Memory Coalescing:
CUDA kernel time: 303.7994 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 2 - Global Memory Coalescing:
CUDA kernel time: 316.1733 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 3 - Global Memory Coalescing:
CUDA kernel time: 300.9598 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 4 - Global Memory Coalescing:
CUDA kernel time: 312.6178 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 5 - Global Memory Coalescing:
CUDA kernel time: 299.2266 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 6 - Global Memory Coalescing:
CUDA kernel time: 316.6690 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 7 - Global Memory Coalescing:
CUDA kernel time: 303.7657 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 8 - Global Memory Coalescing:
CUDA kernel time: 319.0854 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 9 - Global Memory Coalescing:
CUDA kernel time: 304.8051 ms
Results match

### **c. Generate report for the kernel**

In [30]:
!ncu -f --set full -o ./profiles/02_memory_coalesced_gemm ./src/02_memory_coalesced_gemm 2 4096 4096 4096

==PROF== Connected to process 20090 (/home/darshith/code/cuda-gemm-optimization/src/02_memory_coalesced_gemm)
==PROF== Profiling "ampere_sgemm_128x64_nn" - 0: 0%
==WARNING== Launching the workload is taking more time than expected. If this continues to hang, terminate the profile and re-try by profiling the range of all related launches using '--replay-mode range'. See https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#replay for more details.
....50%....100% - 38 passes
Global Memory Coalescing:
==PROF== Profiling "gemmMemCoalesced" - 1: 0%
==WARNING== Launching the workload is taking more time than expected. If this continues to hang, terminate the profile and re-try by profiling the range of all related launches using '--replay-mode range'. See https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#replay for more details.
....50%....100% - 38 passes
CUDA kernel time: 139765.4219 ms
Results match : Yes 
==PROF== Disconnected from process 20090
==PROF== Report

### **d. Report**

**Speed of Light Throughput:**

<div style="text-align: center;">
  <img src="./images/gmem_coalesce_sol.png" alt="Global Memory Coalesce SOL" width="1500">
</div>

Compared to the naive version's heavily memory-bound performance (~90% memory, ~15% compute SOL), this optimized implementation shows balanced utilization with both compute and memory throughput reaching around 85% SOL. This balance, achieved through memory coalescing, indicates more efficient use of both computational and memory resources on the GPU.

**Roofline:**

<div style="text-align: center;">
  <img src="./images/gmem_coalesce_roofline.png" alt="Global Memory Coalesce Roofline" width="1500">
</div>

Compared to the naive implementation (51.8 GFLOP/s), this optimized version achieves 374.9 GFLOP/s with a slightly higher arithmetic intensity of 15.14 FLOP/byte, demonstrating a 7.2x speedup primarily through global memory coalescing, where adjacent threads access contiguous memory addresses to combine memory transactions into fewer, larger operations, significantly reducing memory latency and increasing effective bandwidth.

**Memory Workload Analysis:**

<div style="text-align: center;">
  <img src="./images/gmem_coalesce_mwa.png" alt="Global Memory Coalesce - Memory Workload Analysis" width="2000">
</div>

 The coalesced version shows significant improvements in memory efficiency. Memory throughput increased from 3.56 GB/s to 24.80 GB/s (~7x improvement), and memory utilization per sector improved from 4.0 to 26.4 bytes out of 32 bytes per thread. Memory pipe utilization also increased from 12% to 86.91%. While both kernels maintain high L1TEX hit rates (>94%), the coalesced version's better memory access patterns result in substantially higher bandwidth utilization and overall memory throughput, demonstrating the importance of proper memory coalescing in GEMM implementations.

<div style="text-align: center;">
  <img src="./images/gmem_coalesce_memory_chart.png" alt="Global Memory Coalesce - Memory Chart" width="1000">
</div>

The chart shows 4.30G memory transactions being fetched from global memory. Memory access latencies: global memory takes 400-800 cycles while shared memory only takes 20-30 cycles. Implementing shared memory would significantly improve performance by reducing these high-latency global memory accesses, especially for data that's frequently reused within thread blocks.

## **3 -  Shared Memory Cache-Blocking**

<div style="text-align: center;">
  <img src="./images/shared_memory_cache_blocking.png" alt="Shared Memory Cache Blocking" width="800">
</div>

This diagram illustrates a block-based matrix multiplication algorithm with a block size of 32. When multiplying matrices A and C, each matrix is divided into blocks of size BLOCK_SIZE (32). The starting addresses of blocks are calculated using formulas: `&A = row * BLOCK_SIZE * K` for matrix A's rows, `&B = col * BLOCK_SIZE` for B's columns, and `&C = (row * BLOCK_SIZE * K) + (col * BLOCK_SIZE)` for matrix C's position. As the algorithm processes each block, it increments A by BLOCK_SIZE within the same row, B by BLOCK_SIZE * N to move to the next block, and C moves to process the next row of blocks.

### **a. Compile the code**

The file being executed is [./src/03_shared_memory_gemm.cu](./src/03_shared_memory_gemm.cu)

In [31]:
!nvcc -lineinfo -g -o ./src/03_shared_memory_gemm ./src/run.cu -lcublas -lnvToolsExt

### **b. Execute the object file multiple times**

In [32]:
shared_memory_gemm_time = run_gemm("./src/03_shared_memory_gemm", 3, 4096, 4096, 4096)
print(f"Average Shared Memory GEMM time: {shared_memory_gemm_time}")

Output 1 - Shared Memory Cache-Blocking:
CUDA kernel time: 210.4516 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 2 - Shared Memory Cache-Blocking:
CUDA kernel time: 212.3277 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 3 - Shared Memory Cache-Blocking:
CUDA kernel time: 200.9774 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 4 - Shared Memory Cache-Blocking:
CUDA kernel time: 176.8015 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 5 - Shared Memory Cache-Blocking:
CUDA kernel time: 190.5741 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 6 - Shared Memory Cache-Blocking:
CUDA kernel time: 200.9261 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 7 - Shared Memory Cache-Blocking:
CUDA kernel time: 215.5029 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 8 - Shared Memory Cache-Blocking:
CUDA kernel time: 192.1073 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 9 - Shared Memory Cache-Blocking:
CUDA ke

### **c. Generate report for the kernel**

In [33]:
!ncu -f --set full -o ./profiles/03_shared_memory_gemm ./src/03_shared_memory_gemm 3 4096 4096 4096

==PROF== Connected to process 20800 (/home/darshith/code/cuda-gemm-optimization/src/03_shared_memory_gemm)
==PROF== Profiling "ampere_sgemm_128x64_nn" - 0: 0%
==WARNING== Launching the workload is taking more time than expected. If this continues to hang, terminate the profile and re-try by profiling the range of all related launches using '--replay-mode range'. See https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#replay for more details.
....50%....100% - 38 passes
Shared Memory Cache-Blocking:
==PROF== Profiling "gemmSharedMem" - 1: 0%
==WARNING== Launching the workload is taking more time than expected. If this continues to hang, terminate the profile and re-try by profiling the range of all related launches using '--replay-mode range'. See https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#replay for more details.
....50%....100% - 38 passes
CUDA kernel time: 64080.3320 ms
Results match : Yes 
==PROF== Disconnected from process 20800
==PROF== Report: /

### **d. Report**

**Speed of Light Throughput:**

<div style="text-align: center;">
  <img src="./images/shared_mem_sol.png" alt="Shared Memory SOL" width="1500">
</div>

This shared memory version shows slightly lower GPU throughput (~80% SOL) compared to the coalesced version (~85% SOL), but still maintains balanced compute and memory utilization. It leverages faster shared memory access to reduce global memory traffic, though the overhead of shared memory operations may explain the small performance difference.

**Roofline:**

<div style="text-align: center;">
  <img src="./images/shared_mem_roofline.png" alt="Shared Memory Roofline" width="1500">
</div>

This shared memory implementation reaches 505.0 GFLOP/s compared to 374.9 GFLOP/s in the previous version, with a slightly higher arithmetic intensity (15.85 vs 15.14 FLOP/byte). The 130.1 GFLOP/s improvement comes from optimized tile sizes and shared memory access patterns while maintaining ~80% SOL utilization. Arithmetic Intensity can be increased by having each thread perform operations on multiple locations of output matrix.

**Memory Workload Analysis:**

<div style="text-align: center;">
  <img src="./images/smem_block_tiling_memory_chart.png" alt="Shared Memory Roofline" width="1000">
</div>

Comparing the two memory charts:

Global Memory Only:
- 4.30G memory transactions through global memory
- No shared memory utilization

Shared Memory with Block Tiling:
- Global memory transactions reduced to 268.44M (~ 16x reduction)
- 2.82G transactions now using shared memory
- Shared memory's 20-30 cycle latency vs global memory's 400-800 cycles means significant performance improvement
- More efficient data reuse through shared memory block tiling, reducing high-latency global memory accesses

The dramatic reduction in global memory transactions and shift to faster shared memory access demonstrates the performance benefits of block tiling implementation.

Comparing memory access latencies across the hierarchy:
- Registers: ~1 cycle
- Shared Memory: 20-30 cycles 
- Global Memory: 400-800 cycles

The current block tiling implementation shows good performance by moving from global to shared memory. Further optimization using register blocking would improve performance even more by leveraging single-cycle register access latency compared to shared memory's 20-30 cycles.

**Warp State Statistics:**

<div style="text-align: center;">
  <img src="./images/smem_warp_state_statistics.png" alt="Shared Memory Warp State Statistics" width="2000">
</div>

<div style="text-align: center;">
  <img src="./images/smem_warp_state_chart.png" alt="Shared Memory Warp State Chart" width="2000">
</div>

Analysis of shared memory GEMM kernel performance:

1. Primary bottleneck: MIO Throttle stalls (22.9 cycles/warp)
- 55.8% of total stall time
- Indicates high memory pipeline pressure and shared memory contention

2. Secondary stalls:
- Long Scoreboard: Long-latency operation dependencies 
- Not Selected: Warp scheduling issues

Key metrics:
- 41.02 cycles per issued instruction
- 32 active warps
- 31.99 not predicated off threads per warp

This indicates significant memory-bound execution with warp scheduling constraints impacting throughput.

Increasing arithmetic intensity through tiling would help address the MIO throttle stalls. By increasing computation per memory access ratio, we can better amortize the memory bandwidth bottleneck that's currently causing 55.8% of stall cycles.

## **4 - 1D Block-Tiling**

<div style="text-align: center;">
  <img src="./images/1d_block_tiling.png" alt="1D Block tiling" width="800">
</div>

The parameter `TM` in the kernel specifies the number of rows in the output tile of \( C \) that each thread computes. Instead of handling a single element, each thread processes \( TM \) rows for its assigned column, storing partial results in local registers (`threadResult[TM]`). This approach increases the amount of computation per thread, reducing idle cycles and improving efficiency. By reusing the same tiles of \( A \) and \( B \) loaded into shared memory to compute all \( TM \) rows, the kernel minimizes global memory traffic, enhancing memory locality. This design choice significantly boosts arithmetic intensity by performing more computations relative to the data moved from global memory, leading to better utilization of GPU resources and improved performance.

### **a. Compile the code**

The file being executed is [./src/04_1d_block_tiling.cu](./src/04_1d_block_tiling.cu)

In [34]:
!nvcc -lineinfo -g -o ./src/04_1d_block_tiling ./src/run.cu -lcublas -lnvToolsExt

### **b. Execute the object file multiple times**

In [35]:
block_tile1d_gemm_time = run_gemm("./src/04_1d_block_tiling", 4, 4096, 4096, 4096)
print(f"Average 1D Block-tiled GEMM time: {block_tile1d_gemm_time}")

Output 1 - 1D Block tiling:
CUDA kernel time: 87.4444 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 2 - 1D Block tiling:
CUDA kernel time: 82.2685 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 3 - 1D Block tiling:
CUDA kernel time: 88.1489 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 4 - 1D Block tiling:
CUDA kernel time: 82.6993 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 5 - 1D Block tiling:
CUDA kernel time: 89.2808 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 6 - 1D Block tiling:
CUDA kernel time: 82.9332 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 7 - 1D Block tiling:
CUDA kernel time: 82.4858 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 8 - 1D Block tiling:
CUDA kernel time: 89.3643 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 9 - 1D Block tiling:
CUDA kernel time: 79.4880 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 10 - 1D Block tiling:
CUDA kernel time: 82.4828 

### **c. Generate report for the kernel**

In [36]:
!ncu -f --set full -o ./profiles/04_1d_block_tiling ./src/04_1d_block_tiling 4 4096 4096 4096

==PROF== Connected to process 21323 (/home/darshith/code/cuda-gemm-optimization/src/04_1d_block_tiling)
==PROF== Profiling "ampere_sgemm_128x64_nn" - 0: 0%
==WARNING== Launching the workload is taking more time than expected. If this continues to hang, terminate the profile and re-try by profiling the range of all related launches using '--replay-mode range'. See https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#replay for more details.
....50%....100% - 38 passes
1D Block tiling:
==PROF== Profiling "gemm1dBlockTiling" - 1: 0%....50%....100% - 38 passes
CUDA kernel time: 18777.1621 ms
Results match : Yes 
==PROF== Disconnected from process 21323
==PROF== Report: /home/darshith/code/cuda-gemm-optimization/./profiles/04_1d_block_tiling.ncu-rep


### **d. Report**

**Speed of Light Throughput:**

<div style="text-align: center;">
  <img src="./images/1d_tiling_sol.png" alt="1D tiling SOL" width="1500">
</div>

This 1D tiling implementation achieves ~80% SOL for both compute and memory utilization, compared to the coalesced global memory version which had ~85% SOL. The slight decrease suggests room for further optimization in the tiling strategy.

**Roofline:**

<div style="text-align: center;">
  <img src="./images/1d_tiling_roofline.png" alt="1D Tiling Roofline" width="1500">
</div>

The 1D tiling implementation achieves significantly higher performance at 1,603.9 GFLOP/s compared to the shared memory version's 505.0 GFLOP/s, with nearly doubled arithmetic intensity (28.46 vs 15.85 FLOP/byte). This major improvement suggests much better memory reuse through the tiling strategy.

**Memory Workload Analysis:**

<div style="text-align: center;">
  <img src="./images/1d_tiling_mwa.png" alt="1D Tiling - Memory Work Loan" width="2000">
</div>

This memory workload analysis shows:
- Memory throughput: 56.09 GB/s
- Very low L1TEX hit rate (0.78%), indicating minimal cache reuse
- High memory pipe utilization (81.30%)
- High memory bus utilization (69.07% Mem Busy)

The low cache hit rate but high throughput suggests this kernel is effectively streaming data through memory without relying on cache, which is expected for a 1D tiled GEMM implementation utilizing register-level data reuse.

<div style="text-align: center;">
  <img src="./images/1d_tiling_memory_chart.png" alt="1D Tiling Memory Chart" width="1000">
</div>

Looking at the memory chart, shared memory transactions reduced from 2.82G to 872.42M in this 1D tiled implementation because:
1. Matrix B's values are stored in registers for reuse
2. Matrix C uses register accumulation
3. Each thread computes multiple outputs, increasing data reuse since the same input values can be used for multiple computations before needing new data from shared memory

This thread-level data reuse through registers significantly reduces the frequency of shared memory accesses.

**Warp State Statistics:**

<div style="text-align: center;">
  <img src="./images/1d_tiling_warp_state_statistics.png" alt="1D Tiling Warp State Statistics" width="2000">
</div>

<div style="text-align: center;">
  <img src="./images/1d_tiling_warp_state_chart.png" alt="1D Tiling Warp State Chart" width="2000">
</div>

Comparing the warp state metrics between baseline and 1D tiled implementation:

1. MIO Throttle stalls reduced:
- Baseline: 22.9 cycles (55.8% of stalls)
- 1D Tiled: 7.1 cycles (34.3% of stalls)

2. Overall latency improved:
- Baseline: 41.02 cycles per instruction
- 1D Tiled: 20.84 cycles per instruction

The 1D tiling optimization significantly reduced memory pipeline pressure and improved instruction throughput by ~2x, validating that increased arithmetic intensity helps address the memory bottleneck.

## **5 - 2D Block-Tiling**

### **Loading memory:**

<div style="text-align: center;">
  <img src="./images/stided_memory_load.png" alt="Strided Memory Load" width="800">
</div>

### **2D Tiling**:

<div style="text-align: center;">
  <img src="./images/2d_block_tiling.png" alt="2D Block tiling" width="800">
</div>

This illustration demonstrates 2D tiling in the GEMM kernel, where the computation is partitioned into rectangular tiles. Each block of threads works on a submatrix of C (output matrix) of size BM × BN, while shared memory tiles A and B are loaded as BM × BK and BK × BN, respectively.

Within each tile, threads compute a grid of results (TM × TN), leveraging shared memory for efficient data reuse during multiple dot-product calculations. This approach minimizes global memory accesses and ensures that each thread contributes to a subgrid of C, enhancing computational efficiency and performance through better locality and parallelism.


### **a. Compile the code**

The file being executed is [./src/05_2d_block_tiling.cu](./src/05_2d_block_tiling.cu)

In [37]:
! nvcc -lineinfo -g -o ./src/05_2d_block_tiling ./src/run.cu -lcublas -lnvToolsExt

### **b. Execute the object file multiple times**

In [38]:
block_tile2d_gemm_time = run_gemm("./src/05_2d_block_tiling", 5, 4096, 4096, 4096)
print(f"Average 2D Block-tiled GEMM time: {block_tile2d_gemm_time}")

Output 1 - 2D Block tiling:
CUDA kernel time: 443.6647 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 2 - 2D Block tiling:
CUDA kernel time: 408.4429 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 3 - 2D Block tiling:
CUDA kernel time: 411.0518 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 4 - 2D Block tiling:
CUDA kernel time: 430.6472 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 5 - 2D Block tiling:
CUDA kernel time: 423.2078 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 6 - 2D Block tiling:
CUDA kernel time: 421.6308 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 7 - 2D Block tiling:
CUDA kernel time: 462.7988 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 8 - 2D Block tiling:
CUDA kernel time: 397.8409 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 9 - 2D Block tiling:
CUDA kernel time: 430.2588 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 10 - 2D Block tiling:
CUDA kernel time:

### **c. Generate report for the kernel**

In [39]:
! ncu -f --set full -o ./profiles/05_2d_block_tiling ./src/05_2d_block_tiling 5 4096 4096 4096

==PROF== Connected to process 21717 (/home/darshith/code/cuda-gemm-optimization/src/05_2d_block_tiling)
==PROF== Profiling "ampere_sgemm_128x64_nn" - 0: 0%
==WARNING== Launching the workload is taking more time than expected. If this continues to hang, terminate the profile and re-try by profiling the range of all related launches using '--replay-mode range'. See https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#replay for more details.
....50%....100% - 38 passes
2D Block tiling:
==PROF== Profiling "gemm2dBlockTiling" - 1: 0%
==WARNING== Launching the workload is taking more time than expected. If this continues to hang, terminate the profile and re-try by profiling the range of all related launches using '--replay-mode range'. See https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#replay for more details.
....50%....100% - 38 passes
CUDA kernel time: 64246.9531 ms
Results match : Yes 
==PROF== Disconnected from process 21717
==PROF== Report: /home/darshit

### **d. Report**

**Speed of Light Throughput:**

<div style="text-align: center;">
  <img src="./images/2d_tiling_sol.png" alt="2D tiling SOL" width="1500">
</div>

The 2D tiling's poor performance (15% compute, 55% memory SOL) stems from unaligned global memory access patterns. Unlike 1D tiling's aligned access achieving 80% SOL, misaligned memory transactions in 2D tiling cause memory bandwidth underutilization and thread warp inefficiency.

**Roofline:**

<div style="text-align: center;">
  <img src="./images/2d_tiling_roofline.png" alt="2D Tiling Roofline" width="1500">
</div>

The 2D tiling shows significantly lower performance at 248.7 GFLOP/s versus 1D tiling's 1,603.9 GFLOP/s, with much lower arithmetic intensity (4.76 vs 28.46 FLOP/byte). This stark difference reflects the impact of unaligned memory accesses in the 2D implementation compared to 1D's optimized memory patterns.

**Memory Workload Analysis:**

<div style="text-align: center;">
  <img src="./images/2d_tiling_mwa.png" alt="2D Tiling Memory Workload Analysis" width="1500">
</div>

Comparing 1D vs 2D tiling memory analysis:

1D Tiling:
- Memory throughput: 56.09 GB/s
- Very low L1TEX hit rate (0.78%)
- High memory pipe utilization (81.30%)

2D Tiling:
- Memory throughput: 52.09 GB/s
- Much higher L1TEX hit rate (95.46%)
- Lower memory pipe utilization (19.60%)
- Shows significant bank conflicts in shared memory:
  - 3.2-way bank conflicts for loads (50% of wavefronts)
  - 1.2-way bank conflicts for stores (14.05% of wavefronts)

2D tiling trades slightly lower throughput for better cache utilization, but introduces shared memory bank conflicts that weren't present in 1D tiling. These bank conflicts could be limiting the potential performance gains from 2D tiling.

<div style="text-align: center;">
  <img src="./images/2d_tiling_memory_chart.png" alt="2D Tiling Memory Chart" width="1000">
</div>

Comparing 1D vs 2D tiling memory charts:

1D Tiling:
- Global: 68.16M transactions
- Shared: 872.42M transactions

2D Tiling:
- Global: 570.43M transactions
- Shared: 369.10M transactions (↓57% from 1D)
- Higher cache hit rates (L1: 95.46%, L2: 98.79%)

2D tiling significantly reduces shared memory transactions due to better data reuse through registers, though it shows higher global memory transactions. The improved cache hit rates suggest better memory locality compared to 1D tiling.

## **6 - Vectorized 2D Block-Tiling**

<div style="text-align: center;">
  <img src="./images/vectorized_2d_block_tiling.png" alt="Vectorized 2D Block tiling" width="800">
</div>

Transposing the As matrix transforms the memory access pattern from vertical (strided) to horizontal (contiguous), enabling better memory coalescing. When threads in a warp access consecutive memory locations after transposition, rather than strided locations in the original format, the GPU can fetch data in fewer memory transactions. This is depicted in the diagram where the dotted line shows "Now As matrix can be vectorized as well," illustrating how the transposed layout allows for efficient vectorized memory access patterns, ultimately improving memory bandwidth utilization and kernel performance.

### **a. Compile the code**

The file being executed is [./src/06_vectorize_gemm.cu](./src/06_vectorize_gemm.cu)

In [40]:
!nvcc -lineinfo -g -o ./src/06_vectorize_gemm ./src/run.cu -lcublas -lnvToolsExt

### **b. Execute the object file multiple times**

In [41]:
vector_block_tile2d_gemm_time = run_gemm("./src/06_vectorize_gemm", 6, 4096, 4096, 4096)
print(f"Average Vectroized 2D Block-tiled GEMM time: {vector_block_tile2d_gemm_time}")

Output 1 - Vector - 2D Block tiling:
CUDA kernel time: 23.9152 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 2 - Vector - 2D Block tiling:
CUDA kernel time: 25.7406 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 3 - Vector - 2D Block tiling:
CUDA kernel time: 29.3688 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 4 - Vector - 2D Block tiling:
CUDA kernel time: 28.4600 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 5 - Vector - 2D Block tiling:
CUDA kernel time: 29.9488 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 6 - Vector - 2D Block tiling:
CUDA kernel time: 41.6347 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 7 - Vector - 2D Block tiling:
CUDA kernel time: 31.2112 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 8 - Vector - 2D Block tiling:
CUDA kernel time: 23.9081 ms
Results match : Yes 

Sleeping for 2 seconds...
Output 9 - Vector - 2D Block tiling:
CUDA kernel time: 27.8329 ms
Results match : Yes 



### **c. Generate report for the kernel**

In [42]:
!ncu -f --set full -o ./profiles/06_vectorize_gemm ./src/06_vectorize_gemm 6 4096 4096 4096

==PROF== Connected to process 22210 (/home/darshith/code/cuda-gemm-optimization/src/06_vectorize_gemm)
==PROF== Profiling "ampere_sgemm_128x64_nn" - 0: 0%
==WARNING== Launching the workload is taking more time than expected. If this continues to hang, terminate the profile and re-try by profiling the range of all related launches using '--replay-mode range'. See https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#replay for more details.
....50%....100% - 38 passes
Vector - 2D Block tiling:
==PROF== Profiling "gemmVec2dBlockTiling" - 1: 0%....50%....100% - 38 passes
CUDA kernel time: 5271.5288 ms
Results match : Yes 
==PROF== Disconnected from process 22210
==PROF== Report: /home/darshith/code/cuda-gemm-optimization/./profiles/06_vectorize_gemm.ncu-rep


### **d. Report**

**Speed of Light Throughput:**

<div style="text-align: center;">
  <img src="./images/vector_2d_tiling_sol.png" alt="Vector 2D tiling SOL" width="1500">
</div>

This optimized 2D tiling implementation shows significantly higher utilization with ~70% compute and ~85% memory SOL, compared to the previous 2D tiling's poor 15% compute and 55% memory utilization. The improvement comes from vectorized memory operations that ensure aligned memory accesses.

**Roofline:**

<div style="text-align: center;">
  <img src="./images/vector_2d_tiling_roofline.png" alt="Vector 2D Tiling Roofline" width="1500">
</div>

The vectorized 2D tiling achieves 3,824.1 GFLOP/s with 58.25 FLOP/byte intensity, a dramatic improvement over the basic 2D tiling's 248.7 GFLOP/s and 4.76 FLOP/byte. This 15x performance gain comes from properly aligned vectorized memory access patterns.

**Memory Workload Analysis:**

<div style="text-align: center;">
  <img src="./images/vector_2d_tiling_mwa.png" alt="Vector 2D Tiling Memory Workload Analysis" width="1500">
</div>

Comparing vectorized vs regular 2D tiling:

- Memory Performance:
    - Throughput increased: 52.09 → 65.64 GB/s
    - Memory bus utilization improved: 47.13% → 83.55%

- Bank Conflicts:
    - Previous: 3.2-way load conflicts (50%)
    - Current: 5.0-way load conflicts (40%)

- Global Memory:
    - Previous: 4.4/32 bytes utilized per sector
    - Current: 31.5/32 bytes utilized per sector (significant coalescing improvement)

The vectorized implementation shows better memory throughput and global memory coalescing, though with slightly higher bank conflicts in shared memory.

<div style="text-align: center;">
  <img src="./images/vector_2d_tiling_memory_chart.png" alt="Vector 2D Tiling Memory Chart" width="1000">
</div>

Comparing memory charts for vectorized 2D tiling:

- Previous:
    - Global: 570.43M transactions
    - Shared: 369.10M transactions

- Current:
    - Global: 8.65M transactions (↓98%)
    - Shared: 155.19M transactions (↓58%)
    - Significant reduction in total memory operations due to vectorization's efficient data loading and register usage

Both L1 and L2 cache hit rates remain similar, but the dramatic reduction in memory transactions shows vectorization's effectiveness at memory access optimization.

**Warp State Statistics:**

<div style="text-align: center;">
  <img src="./images/vector_2d_tiling_warp_state_statistics.png" alt="Vector 2D Tiling Warp State Statistics" width="2000">
</div>

<div style="text-align: center;">
  <img src="./images/vector_2d_tiling_warp_state_chart.png" alt="Vector 2D Tiling Warp State Chart" width="2000">
</div>

Comparing metrics across implementations:

Baseline:
- 41.02 cycles/instruction
- MIO throttle: 22.9 cycles (55.8%)

1D Tiled:
- 20.84 cycles/instruction
- MIO throttle: 7.1 cycles (34.3%)

Vectorized 2D Tiled:
- 6.16 cycles/instruction
- MIO throttle further reduced
- Main bottleneck shifted to Not Selected stalls

2D tiling with vectorization achieved ~6.7x speedup vs baseline and ~3.4x vs 1D tiling, showing effective memory pressure reduction through increased arithmetic intensity.